In [1]:
from burstextractor.burstlist import download_burst_data
from burstextractor.timeutils import extract_time, fix_typos_in_time, fix_24_hour_time, create_datetime
from burstextractor.data_utils import explode_instruments_long_clean_instruments, keep_only_type_I_to_VI
import pandas as pd
import numpy as np
from database_utils import extract_instrument_name

## Create Dataframe with the Burstlists

In [2]:
burst_list = download_burst_data([2021, 2022, 2023], months=range(1, 13), folder="ecallisto_files")

/home/vincenzo/sunflare_prediction/burstextractor/burstlist.py:39: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  data = pd.read_csv(
/home/vincenzo/sunflare_prediction/burstextractor/burstlist.py:39: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  data = pd.read_csv(


In [3]:
burst_list

,date,time,type,instruments
0,20210119,02:42-02:42,III,Australia-ASSA
1,20210120,12:37-12:37,III,"AUSTRIA-UNIGRAZ, [HUMAIN], MRT1, SOUTHAFRICA-S..."
2,20210127,04:32-04:32,III,"Australia-ASSA, INDIA-GAURI, SOUTHAFRICA-SANSA"
3,20210127,09:27-09:27,III,"AUSTRIA-UNIGRAZ, INDIA-GAURI, INDIA-OOTY, MRT1..."
4,20210218,18:04-18:04,III,"GREENLAND, MEXART, ROSWELL-NM"
...,...,...,...,...
5354,20230224,04:16-04:25,III,"Australia-ASSA, INDIA-OOTY, INDIA-UDAIPUR, IND..."
5355,20230224,06:31-06:33,III,"Australia-ASSA, INDIA-OOTY, INDIA-UDAIPUR, SSRT"
5356,20230224,06:56-06:57,III,"Australia-ASSA, INDIA-OOTY, (SSRT)"
5357,20230224,07:49-07:50,III,"Australia-ASSA, EGYPT-Alexandria, GERMANY-DLR,..."


In [4]:
burst_list.sample(5)

,date,time,type,instruments
2144,20220414,16:13-16:14,III,"Arecibo-Observatory, GREENLAND, HUMAIN, ROSWEL..."
5135,20230208,04:21-04:44,VI,"Australia-ASSA, SSRT"
1271,20211228,06:37-06:47,VI,"Australia-ASSA, INDIA-UDAIPUR"
5091,20230203,17:43-17:43,III,"Arecibo-Observatory, MEXICO-LANCE, USA-ARIZONA..."
3667,20220912,11:31-12:08,CTM,GLASGOW


In [5]:
burst_list.shape

(5359, 4)

## Fix typos

In [6]:
extracted_digits = burst_list['time'].str.extract(r'(\d+).(\d+).(\d+).(\d+)', expand=True).astype(int)

In [7]:
impossible_times_bool = (extracted_digits[1] > 59) | (extracted_digits[3] > 59) | (extracted_digits[0] > 23) | (extracted_digits[2] > 23)
extracted_digits[impossible_times_bool]

,0,1,2,3
1693,6,6,6,88
2238,24,32,14,33
3876,23,59,24,0
3890,0,0,24,0
3920,3,50,24,0
4179,23,59,24,0
4467,23,58,24,0
4557,0,0,24,0
4576,0,0,24,0


In [8]:
burst_list[impossible_times_bool]

,date,time,type,instruments
1693,20220210,06:06-06:88,V,"ALMATY, Australia-ASSA, INDIA-OOTY, INDIA-UDAI..."
2238,20220421,24:32-14:33,III,"AUSTRIA-MICHELBACH, Arecibo-Observatory, GLASG..."
3876,20220926,23:59:24:00,III,"ALASKA-COHOE, ALASKA-HAARP, Australia-ASSA"
3890,20220929,00:00-24:00,NaN,CTM
3920,20221001,03:50-24:00,CTM,*
4179,20221110,23:59-24:00,III,"ALASKA-COHOE, Australia-ASSA"
4467,20221215,23:58-24:00,III,Arecibo-Observatory
4557,20221221,00:00-24:00,CTM,*
4576,20221222,00:00-24:00,CTM,*


In [9]:
burst_list.loc[4179]

date                               20221110
time                            23:59-24:00
type                                    III
instruments    ALASKA-COHOE, Australia-ASSA
Name: 4179, dtype: object

In [10]:
burst_list.loc[4179+1]

date                 20221111
time              00:02-00:14
type                       VI
instruments    Australia-ASSA
Name: 4180, dtype: object

In [11]:
burst_list = fix_typos_in_time(burst_list)
burst_list = extract_time(burst_list)
burst_list = fix_24_hour_time(burst_list)
burst_list = create_datetime(burst_list)
burst_list = explode_instruments_long_clean_instruments(burst_list)
burst_list = keep_only_type_I_to_VI(burst_list)

In [12]:
burst_list.sample(5)

,date,time,type,instruments,time_start,time_end,date_start,date_end,datetime_start,datetime_end
22145,20221226,01:25-01:36,VI,Australia-ASSA,01:25,01:36,20221226,20221226,2022-12-26 01:25:00,2022-12-26 01:36:00
18636,20220926,12:13-12:13,III,TRIEST,12:13,12:13,20220926,20220926,2022-09-26 12:13:00,2022-09-26 12:13:00
2314,20210823,11:46-11:46,III,DENMARK,11:46,11:46,20210823,20210823,2021-08-23 11:46:00,2021-08-23 11:46:00
21463,20221216,13:19-13:19,III,GERMANY-DLR,13:19,13:19,20221216,20221216,2022-12-16 13:19:00,2022-12-16 13:19:00
22852,20230108,13:50-13:53,III,AUSTRIA-UNIGRAZ,13:50,13:53,20230108,20230108,2023-01-08 13:50:00,2023-01-08 13:53:00


## Fix wrong names

In [13]:
MIN_BURST_PER_INSTRUMENT = 30

In [14]:
low_appearance_instruments = burst_list.groupby('instruments').filter(lambda x: len(x) <= MIN_BURST_PER_INSTRUMENT).instruments.unique()
low_appearance_instruments

array(['INDIA-UAIPUR', 'SPAIN-ALCALA', 'ROSWELL-NW', 'Australia-LMRO', '',
       'INDIA-NASHIK', 'DENMARK. GLASGOW', 'SWISS-BLEN5M', 'SWISS-BLEN7M',
       'URUGUAY', 'INPE?', 'MRT', 'SWISS-BLEN7M-E', 'NORWAY-RANDABERG',
       'INDIA-GAURI?', 'MRO?', 'AUSTRTIA-MICHELBACH', '/INDIA-UDAIPUR',
       'l MONGOLIA-UB', 'SWISS-Landschlach', 'HUMAIN. SWISS-Landschlacht',
       'GASGOW', 'UNAM', 'GLSAGOW', 'INDOENSIA', 'DENMAARK', 'Humain',
       'SWISS-LandschlachtEGYPT-Alexandria', 'INDIA-UDAIPUR MRT1',
       'AUSTRIA-Krumbach', 'NORWAY-NY-AALESUND', 'SP', 'IAIN-PERALEJOS',
       'ALASKA-ANCHORAGE', 'HUAMAIN', 'NDIA-GAURI', 'ROMANIA', 'HUMAIn',
       'MRT1?', 'HUMAI', 'NDIA-UDAIPUR', 'LASKA-COHOE', 'NDIA-OOTY',
       'MEXARFT', 'POLAND', 'USTRIA-UNIGRAZ', 'MRT21',
       'THAILAND-Pathumthan', 'INDIAMONGOLIA-UB', 'SSRT-UDAIPUR', '*',
       'Australia-ASSAArecibo-Observatory',
       'SSRT {more like drifting chain of type I}', 'INDIAALMATY',
       'SSRT-GAURI', 'INDOALASKA-COHOE', 

In [15]:
import difflib

In [16]:
def find_closest_instrument(instrument):
    try:
        instruments = burst_list.instruments.unique().tolist()
        instruments = np.setdiff1d(instruments, low_appearance_instruments)
        close_instrument = difflib.get_close_matches(instrument, instruments, n=1, cutoff=0.7)[0]
        print(f"Looking for a close match for {instrument}. Found {close_instrument}")
        return close_instrument
    except IndexError:
        print(f"Could not find a close match for {instrument}. Returning NaN.")
        return pd.NA

In [17]:
burst_list['instruments'] = burst_list['instruments'].apply(lambda x: find_closest_instrument(x) if x in low_appearance_instruments else x)

Looking for a close match for INDIA-UAIPUR. Found INDIA-UDAIPUR
Looking for a close match for INDIA-UAIPUR. Found INDIA-UDAIPUR
Could not find a close match for SPAIN-ALCALA. Returning NaN.
Looking for a close match for ROSWELL-NW. Found ROSWELL-NM
Could not find a close match for SPAIN-ALCALA. Returning NaN.
Looking for a close match for ROSWELL-NW. Found ROSWELL-NM
Looking for a close match for Australia-LMRO. Found Australia-ASSA
Looking for a close match for Australia-LMRO. Found Australia-ASSA
Could not find a close match for . Returning NaN.
Could not find a close match for SPAIN-ALCALA. Returning NaN.
Could not find a close match for SPAIN-ALCALA. Returning NaN.
Could not find a close match for INDIA-NASHIK. Returning NaN.
Could not find a close match for SPAIN-ALCALA. Returning NaN.
Could not find a close match for SPAIN-ALCALA. Returning NaN.
Could not find a close match for DENMARK. GLASGOW. Returning NaN.
Could not find a close match for SPAIN-ALCALA. Returning NaN.
Could no

In [18]:
burst_list[burst_list['instruments'].isna()]

,date,time,type,instruments,time_start,time_end,date_start,date_end,datetime_start,datetime_end
267,20210426,13:56-13:56,III,<NA>,13:56,13:56,20210426,20210426,2021-04-26 13:56:00,2021-04-26 13:56:00
451,20210509,13:54-13:56,III,<NA>,13:54,13:56,20210509,20210509,2021-05-09 13:54:00,2021-05-09 13:56:00
811,20210522,01:00-02:48,VI,<NA>,01:00,02:48,20210522,20210522,2021-05-22 01:00:00,2021-05-22 02:48:00
868,20210522,06:05-06:05,III,<NA>,06:05,06:05,20210522,20210522,2021-05-22 06:05:00,2021-05-22 06:05:00
889,20210522,06:16-06:17,III,<NA>,06:16,06:17,20210522,20210522,2021-05-22 06:16:00,2021-05-22 06:17:00
...,...,...,...,...,...,...,...,...,...,...
25253,20230223,06:12-06:15,V,<NA>,06:12,06:15,20230223,20230223,2023-02-23 06:12:00,2023-02-23 06:15:00
25281,20230223,07:47-07:50,III,<NA>,07:47,07:50,20230223,20230223,2023-02-23 07:47:00,2023-02-23 07:50:00
25288,20230223,07:47-07:50,III,<NA>,07:47,07:50,20230223,20230223,2023-02-23 07:47:00,2023-02-23 07:50:00
25310,20230223,08:08-08:10,III,<NA>,08:08,08:10,20230223,20230223,2023-02-23 08:08:00,2023-02-23 08:10:00


In [19]:
burst_list = burst_list.dropna(subset=['instruments'])

In [20]:
extract_instrument_name('Australia-ASSA')

'australia_assa'

In [21]:
burst_list['database_instrument_name'] = burst_list['instruments'].apply(lambda x: extract_instrument_name(x))

In [22]:
burst_list[burst_list.instruments.str.contains('Australia-ASSA')]

,date,time,type,instruments,time_start,time_end,date_start,date_end,datetime_start,datetime_end,database_instrument_name
0,20210119,02:42-02:42,III,Australia-ASSA,02:42,02:42,20210119,20210119,2021-01-19 02:42:00,2021-01-19 02:42:00,australia_assa
7,20210127,04:32-04:32,III,Australia-ASSA,04:32,04:32,20210127,20210127,2021-01-27 04:32:00,2021-01-27 04:32:00,australia_assa
89,20210419,06:55-06:57,III,Australia-ASSA,06:55,06:57,20210419,20210419,2021-04-19 06:55:00,2021-04-19 06:57:00,australia_assa
108,20210419,23:39-23:42,II,Australia-ASSA,23:39,23:42,20210419,20210419,2021-04-19 23:39:00,2021-04-19 23:42:00,australia_assa
109,20210419,23:39-23:43,III,Australia-ASSA,23:39,23:43,20210419,20210419,2021-04-19 23:39:00,2021-04-19 23:43:00,australia_assa
...,...,...,...,...,...,...,...,...,...,...,...
25412,20230224,02:17-03:56,VI,Australia-ASSA,02:17,03:56,20230224,20230224,2023-02-24 02:17:00,2023-02-24 03:56:00,australia_assa
25414,20230224,04:16-04:25,III,Australia-ASSA,04:16,04:25,20230224,20230224,2023-02-24 04:16:00,2023-02-24 04:25:00,australia_assa
25421,20230224,06:31-06:33,III,Australia-ASSA,06:31,06:33,20230224,20230224,2023-02-24 06:31:00,2023-02-24 06:33:00,australia_assa
25425,20230224,06:56-06:57,III,Australia-ASSA,06:56,06:57,20230224,20230224,2023-02-24 06:56:00,2023-02-24 06:57:00,australia_assa


In [23]:
burst_list['type'] = burst_list['type'].replace({'I': 1, 'II': 2, 'III': 3, 'IV': 4, 'V': 5, 'VI': 6}).astype(int)

In [24]:
burst_list[burst_list.instruments.str.contains('Australia-ASSA') & burst_list.type.isin([5, 6])]

,date,time,type,instruments,time_start,time_end,date_start,date_end,datetime_start,datetime_end,database_instrument_name
113,20210420,03:19-03:21,6,Australia-ASSA,03:19,03:21,20210420,20210420,2021-04-20 03:19:00,2021-04-20 03:21:00,australia_assa
168,20210423,07:16-07:16,6,Australia-ASSA,07:16,07:16,20210423,20210423,2021-04-23 07:16:00,2021-04-23 07:16:00,australia_assa
332,20210506,03:00-03:04,6,Australia-ASSA,03:00,03:04,20210506,20210506,2021-05-06 03:00:00,2021-05-06 03:04:00,australia_assa
492,20210512,05:43-05:46,6,Australia-ASSA,05:43,05:46,20210512,20210512,2021-05-12 05:43:00,2021-05-12 05:46:00,australia_assa
534,20210513,06:28-06:30,6,Australia-ASSA,06:28,06:30,20210513,20210513,2021-05-13 06:28:00,2021-05-13 06:30:00,australia_assa
...,...,...,...,...,...,...,...,...,...,...,...
25014,20230221,03:50-04:01,6,Australia-ASSA,03:50,04:01,20230221,20230221,2023-02-21 03:50:00,2023-02-21 04:01:00,australia_assa
25191,20230222,01:14-07:50,6,Australia-ASSA,01:14,07:50,20230222,20230222,2023-02-22 01:14:00,2023-02-22 07:50:00,australia_assa
25225,20230223,00:03-07:00,6,Australia-ASSA,00:03,07:00,20230223,20230223,2023-02-23 00:03:00,2023-02-23 07:00:00,australia_assa
25239,20230223,06:12-06:15,5,Australia-ASSA,06:12,06:15,20230223,20230223,2023-02-23 06:12:00,2023-02-23 06:15:00,australia_assa


In [25]:
burst_list.to_excel('burst_list.xlsx', index=False)

In [26]:
from database_functions import *
from database_utils import *
from tqdm import tqdm
import datetime

In [27]:
def insert_is_burst_status_between_dates_sql(tablename, start_date, end_date, type):
    """Insert is_burst status between two dates.

    Parameters
    ----------
    tablename : str
        The table name to insert the is_burst status for.
    start_date : `~datetime.datetime`
        The start date to insert the is_burst status for.
    end_date : `~datetime.datetime`
        The end date to insert the is_burst status for.
    type : int
        The type of burst to insert the burst_type status for. 


    Returns
    -------
    None

    Notes
    -----
    The function first finds the unique index data and the indices of the non-unique index data.
    It then combines the non-unique index data using the method specified by the `method` parameter.
    """
    start_date = start_date.strftime("%Y-%m-%d %H:%M:%S")
    end_date = end_date.strftime("%Y-%m-%d %H:%M:%S")
    with psycopg2.connect(CONNECTION) as conn:
        cursor = conn.cursor()
        cursor.execute(
            f"""
        UPDATE {tablename}
        SET burst_type = {type}
        WHERE datetime BETWEEN '{start_date}' AND '{end_date}'
        """
        )
        conn.commit()
        cursor.close()

In [28]:
MIN_DATE = datetime.datetime(2022, 1, 1)
burst_list_filtered = burst_list[burst_list['datetime_start'] >= MIN_DATE]

In [29]:
for table in tqdm(get_table_names_sql()):
    specific_instrument = table
    if table[-2:].isnumeric():
        table = table[:-3]
        burst_list_table = burst_list_filtered[burst_list_filtered['database_instrument_name'] == table].copy()
        if not 'burst_type' in get_column_names_sql(specific_instrument):
            add_new_column_default_value_sql(specific_instrument, 'burst_type', 'SMALLINT', 0)
        for i, row in burst_list_table.iterrows():
            insert_is_burst_status_between_dates_sql(specific_instrument, row.datetime_start, row.datetime_end, row.type)
    else:
        raise ValueError(f"Table name {table} is not in the correct format.")


100%|██████████| 3/3 [02:46<00:00, 55.54s/it]
